In [1]:
# Please install the packages in this way :
# It will be directly installed in current enviorenmet 
#import sys
#!{sys.executable} -m pip install pydot
#!{sys.executable} -m pip install sklearn
#!{sys.executable} -m pip install bert-tensorflow

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import pandas as pd
import re
import keras.layers as layers
import pydot

from collections import Counter
from keras import backend as K
from keras.callbacks import TensorBoard
#from tensorflow.python.keras.callbacks import LearningRateScheduler
from keras.callbacks import LearningRateScheduler

from keras.layers import Input, Embedding, BatchNormalization, LSTM, Dense, Concatenate,Bidirectional
from keras.models import Model

from keras.utils import plot_model

W0606 08:03:15.648323 139765616489856 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.


In [3]:
# Reduce TensorFlow logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

# Instantiate the elmo model
elmo_module = hub.Module("https://tfhub.dev/google/elmo/1", trainable=False)

# Initialize session

#K.set_session(sess)

#K.set_learning_phase(1)

#sess.run(tf.global_variables_initializer())
#sess.run(tf.tables_initializer())

In [4]:
## let's import the dataset here
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
#train=train.sample(100000)
#test=test.sample(10000)

In [5]:
train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [6]:
print (len(train))

1804874


In [7]:
## using sklearn for splitting the dataset in train and validation set
## test set has been already provided

from sklearn.model_selection import train_test_split

train, val = train_test_split(train,test_size = 0.05, random_state = 23)    

In [8]:
## Lengh of the validation set
print (len(val))

90244


In [9]:
## This function if for preprocessing the dataset
def preprocess(data):
    '''
    Credit goes to https://www.kaggle.com/gpreda/jigsaw-fast-compact-solution
    '''
    punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
    def clean_special_chars(text, punct):
        for p in punct:
            text = text.replace(p, ' ')
        return text

    data = data.astype(str).apply(lambda x: clean_special_chars(x, punct))
    return data

In [10]:
## checking if GPU is working fine or not

import tensorflow as tf
tf.test.is_gpu_available()

True

In [11]:
# initializing the parameter of max word length, I am taking it 100 to make algorithm faster
time_steps = 100


# building vocabulary from dataset
def build_vocabulary(sentence_list):
    unique_words = " ".join(sentence_list).strip().split()
    word_count = Counter(unique_words).most_common()
    vocabulary = {}
    for word, _ in word_count:
        vocabulary[word] = len(vocabulary)        

    return vocabulary


# Get vocabulary vectors from document list
# Vocabulary vector, Unknown word is 1 and padding is 0
# INPUT: raw sentence list
# OUTPUT: vocabulary vectors list
def get_voc_vec(document_list, vocabulary):    
    voc_ind_sentence_list = []
    for document in document_list:
        voc_idx_sentence = []
        word_list = document.split()
        
        for w in range(time_steps):
            if w < len(word_list):
                # pickup vocabulary id and convert unknown word into 1
                voc_idx_sentence.append(vocabulary.get(word_list[w], -1) + 2)
            else:
                # padding with 0
                voc_idx_sentence.append(0)
            
        voc_ind_sentence_list.append(voc_idx_sentence)
        
    return np.array(voc_ind_sentence_list)


vocabulary = build_vocabulary(train['comment_text'])


In [12]:
## these are the identity columns, 

identity_columns = ['asian', 'atheist',
       'bisexual', 'black', 'buddhist', 'christian', 'female',
       'heterosexual', 'hindu', 'homosexual_gay_or_lesbian',
       'intellectual_or_learning_disability', 'jewish', 'latino', 'male',
       'muslim', 'other_disability', 'other_gender',
       'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white']

y_identities_train = (train[identity_columns] >= 0.5).astype(int).values
y_identities_val = (val[identity_columns] >= 0.5).astype(int).values

In [13]:
def run_labelling(identity_columns,train):  
    '''
    credits go to: https://www.kaggle.com/tanreinama/simple-lstm-using-identity-parameters-solution/ 
    '''
    #identity_columns = ['asian', 'atheist',
    #   'bisexual', 'black', 'buddhist', 'christian', 'female',
    #   'heterosexual', 'hindu', 'homosexual_gay_or_lesbian',
    #   'intellectual_or_learning_disability', 'jewish', 'latino', 'male',
    #   'muslim', 'other_disability', 'other_gender',
    #   'other_race_or_ethnicity', 'other_religion',
    #   'other_sexual_orientation', 'physical_disability',
    #   'psychiatric_or_mental_illness', 'transgender', 'white']
       
    # Overall
    weights = np.ones((len(train),)) / 4
    # Subgroup
    weights += (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) / 4
    # Background Positive, Subgroup Negative
    weights += (( (train['target'].values>=0.5).astype(bool).astype(np.int) +
       (train[identity_columns].fillna(0).values<0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
    # Background Negative, Subgroup Positive
    weights += (( (train['target'].values<0.5).astype(bool).astype(np.int) +
       (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
    loss_weight = 1.0 / weights.mean()
    #y_train = (train['target'].values>=0.5).astype(np.int).T
    y_train = np.vstack([(train['target'].values>=0.5).astype(np.int),weights]).T
    y_aux_train = train[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']].values
    Y_n_t=np.hstack([y_train, y_aux_train])
    return Y_n_t, loss_weight

In [14]:


Y_n_t, loss_weight_train = run_labelling(identity_columns,train)
Y_n_v, loss_weight_val = run_labelling(identity_columns,val)

In [15]:
#del y_train
print(Y_n_t)

[[0.         0.25       0.         ... 0.16666667 0.16666667 0.        ]
 [0.         0.75       0.16666667 ... 0.16666667 0.         0.        ]
 [0.         0.25       0.16666667 ... 0.         0.16666667 0.        ]
 ...
 [0.         0.75       0.         ... 0.         0.         0.        ]
 [0.         0.25       0.         ... 0.         0.         0.        ]
 [0.         0.25       0.         ... 0.         0.         0.        ]]


In [16]:
# mini-batch generator
def batch_iter(data, labels, batch_size, shuffle=False):
    num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1
    print("batch_size", batch_size)
    print("num_batches_per_epoch", num_batches_per_epoch)

    def data_generator():
        data_size = len(data)

        while True:
            # Shuffle the data at each epoch
            if shuffle:
                shuffle_indices = np.random.permutation(np.arange(data_size))
                shuffled_data = data[shuffle_indices]
                shuffled_labels = labels[shuffle_indices]
            else:
                shuffled_data = data
                shuffled_labels = labels

            for batch_num in range(num_batches_per_epoch):
                start_index = batch_num * batch_size
                end_index = min((batch_num + 1) * batch_size, data_size)
                ###
                shuffled_D=shuffled_data[start_index: end_index]
                shuffled_D.tolist()
                shuffled_D = [' '.join(t.split()[0:time_steps]) for t in shuffled_D]
                shuffled_D = np.array(shuffled_D)                
                X_voc = get_voc_vec(shuffled_D, vocabulary)
                                
                sentence_split_list = []
                sentence_split_length_list = []
                
                

                for sentence in shuffled_D:
                    
                    
                    
                    sentence_split = sentence.split()
                    sentence_split_length = len(sentence_split)
                    sentence_split += ["NaN"] * (time_steps - sentence_split_length)
                    
                    sentence_split_list.append((" ").join(sentence_split))
                    sentence_split_length_list.append(sentence_split_length)
        
                X_elmo = np.array(sentence_split_list)

                X = [X_voc, X_elmo]
                
                y = shuffled_labels[start_index: end_index]
                y_train=y[:,[0, 1]]
                y_aux_train=y[:,[2,3,4,5,6,7]]
                
                Lables=[y_train,y_aux_train]
                yield X,Lables 

    return num_batches_per_epoch, data_generator()

In [17]:
# Create datasets (Only take up to time_steps words for memory)

#train_text = train['comment_text'].tolist()
#train_text = [' '.join(t.split()[0:time_steps]) for t in train_text]
#train_text = np.array(train_text)
#train_label = np.array(train_df['polarity'].tolist())
#train_label=(train['target'].values>=0.5).astype(np.int).T

#val_text = val['comment_text'].tolist()
#val_text = [' '.join(t.split()[0:time_steps]) for t in val_text]
#val_text = np.array(val_text)
#train_label = np.array(train_df['polarity'].tolist())
#val_label =(val['target'].values>=0.5).astype(np.int).T




In [18]:
import numpy as np
# mini-batch size
batch_size = 256

train_steps, train_batches = batch_iter(train['comment_text'],
                                        Y_n_t,
                                        batch_size)
valid_steps, valid_batches = batch_iter(val['comment_text'],
                                        Y_n_v,
                                        batch_size)



batch_size 256
num_batches_per_epoch 6698
batch_size 256
num_batches_per_epoch 353


In [19]:
num_aux_targets=Y_n_t.shape[-1]-2

del train,  val,  Y_n_t

In [20]:
# embed elmo method
def make_elmo_embedding(x):
    embeddings = elmo_module(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["elmo"]
    
    return embeddings

In [21]:
from tensorflow.python.keras.losses import binary_crossentropy

#from keras.losses import binary_crossentropy
def custom_loss(y_true, y_pred):
    return binary_crossentropy(K.reshape(y_true[:,0],(-1,1)), y_pred) * y_true[:,1]


In [22]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [23]:
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, Dropout, add, concatenate
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 768

#num_aux_targets=Y_n_t.shape[-1]-2
loss_weight =loss_weight_train

# elmo embedding dimension
elmo_dim = 1024
time_steps = 100

# Input Layers

#word_input = Input(shape=(100,), dtype='int32')  # (batch_size, sent_length)
word_input = Input(shape=(100, ), dtype='int32')  # (batch_size, sent_length)
#elmo_input = Input(shape=(100,), dtype='string')  # (batch_size, sent_length, elmo_size)
elmo_input = Input(shape=(None, ), dtype='string')  # (batch_size, sent_length, elmo_size)


# Hidden Layers
word_embedding = Embedding(input_dim=len(vocabulary), output_dim=128, mask_zero=False)(word_input)
elmo_embedding = layers.Lambda(make_elmo_embedding, output_shape=(100, elmo_dim))(elmo_input)
word_embedding = Concatenate()([word_embedding, elmo_embedding])
word_embedding = BatchNormalization()(word_embedding)

x = SpatialDropout1D(0.3)(word_embedding)
#x = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2,merge_mode='concat')(word_embedding))
#model.add( Bidirectional( LSTM(lstm_out = 196, dropout_U = 0.2, dropout_W = 0.2)))


    
x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)
x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)


att = Attention(100)(x)
hidden = concatenate([att, GlobalMaxPooling1D()(x),GlobalAveragePooling1D()(x)])

hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
result = Dense(1, activation='sigmoid')(hidden)
aux_result = Dense(num_aux_targets, activation='sigmoid')(hidden)
    
model = Model(inputs=[word_input, elmo_input], outputs=[result, aux_result])
model.compile(loss=[custom_loss,'binary_crossentropy'], loss_weights=[loss_weight, 1.0], optimizer='adam')





##dense = Dense(1, activation='sigmoid')(hidden)
# Output Layer
#predict = Dense(units=1, activation='sigmoid')(x)
#pred = layers.Dense(1, activation='sigmoid')(x) #### num Aux target
#aux_result = layers.Dense(num_aux_targets, activation='sigmoid')(x)



#model = Model(inputs=[word_input, elmo_input], outputs=pred)
#model = Model(inputs=[word_input, elmo_input], outputs=[pred,aux_result])
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
#model.compile(loss=[custom_loss,'binary_crossentropy'], loss_weights=[loss_weight,1.0], optimizer='adam', metrics=['accuracy'])
model.summary()

plot_model(model, to_file="model.png", show_shapes=True)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 128)     206828928   input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 100, 1024)    0           input_2[0][0]                    
__________________________________________________________________________________________________
concatenat

In [24]:
def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)

In [25]:
print(valid_batches)

<generator object batch_iter.<locals>.data_generator at 0x7f1cff7015c8>


In [26]:

#from tensorflow.python.keras.callbacks import TensorBoard, LearningRateScheduler
NUM_EPOCHS = 1
logfile_path = './log'
tb_cb = TensorBoard(log_dir=logfile_path, histogram_freq=0)
checkpoint_predictions = []
checkpoint_val_preds = []
weights = []
NUM_MODELS = 1
sess = tf.Session()
initialize_vars(sess)
for model_idx in range(NUM_MODELS):
    for global_epoch in range(NUM_EPOCHS):
        history = model.fit_generator(train_batches, train_steps,
                              epochs=2, 
                              validation_data=valid_batches,
                              validation_steps=valid_steps,
                              #callbacks=[tb_cb]
                              callbacks=[LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** global_epoch))]
                              )
        
        #checkpoint_predictions.append([0].flatten())
        #checkpoint_val_preds.append(model.predict(x_train[val_ind], batch_size=2048)[0].flatten())
        #print(model.predict_generator(valid_batches, steps=valid_steps))
        checkpoint_val_preds.append(model.predict_generator(valid_batches, steps=valid_steps)[0].flatten())
        weights.append(2 ** global_epoch)
        
        #[0].flatten()
        
#

/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:107: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 206828928 elements. This may consume a large amount of memory.
  num_elements)


Epoch 1/2
6698/6698 [==============================] - 14764s 2s/step - loss: 0.3212 - dense_3_loss: 0.0658 - dense_4_loss: 0.1112 - val_loss: 0.3006 - val_dense_3_loss: 0.0606 - val_dense_4_loss: 0.1071
Epoch 2/2
6698/6698 [==============================] - 14775s 2s/step - loss: 0.2478 - dense_3_loss: 0.0453 - dense_4_loss: 0.1034 - val_loss: 0.3255 - val_dense_3_loss: 0.0680 - val_dense_4_loss: 0.1085


In [27]:
print(checkpoint_val_preds)

[array([0.00033218, 0.00017622, 0.00011253, ..., 0.01226839, 0.00314671,
       0.00031343], dtype=float32)]


In [28]:
model.save('ElmoModel.h5')


In [29]:
#before this please rerun the model building steps 
#initialize_vars(sess)
#model.load_weights('BertModel.h5')

In [30]:
val_preds = np.average(checkpoint_val_preds, weights=weights, axis=0)

In [31]:
print(val_preds)
print(len(val_preds))

[0.00033218 0.00017622 0.00011253 ... 0.01226839 0.00314671 0.00031343]
90244


In [32]:
from sklearn.metrics import roc_auc_score

def power_mean(x, p=2):
    return np.power(np.mean(np.power(x, p)),1/p)

def get_s_auc(y_true,y_pred,y_identity):
    mask = y_identity==1
    try:
        s_auc = roc_auc_score(y_true[mask],y_pred[mask])
    except:
        s_auc = 1
    return s_auc

def get_bpsn_auc(y_true,y_pred,y_identity):
    mask = (y_identity==1) & (y_true==0) | (y_identity==0) & (y_true==1)
    try:
        bpsn_auc = roc_auc_score(y_true[mask],y_pred[mask])
    except:
        bpsn_auc = 1
    return bpsn_auc

def get_bspn_auc(y_true,y_pred,y_identity):
    mask = (y_identity==1) & (y_true==1) | (y_identity==0) & (y_true==0)
    try:
        bspn_auc = roc_auc_score(y_true[mask],y_pred[mask])
    except:
        bspn_auc = 1
    return bspn_auc

def get_total_auc(y_true,y_pred,y_identities):

    N = y_identities.shape[1]
    
    saucs = np.array([get_s_auc(y_true,y_pred,y_identities[:,i]) for i in range(N)])
    bpsns = np.array([get_bpsn_auc(y_true,y_pred,y_identities[:,i]) for i in range(N)])
    bspns = np.array([get_bspn_auc(y_true,y_pred,y_identities[:,i]) for i in range(N)])

    M_s_auc = power_mean(saucs)
    M_bpsns_auc = power_mean(bpsns)
    M_bspns_auc = power_mean(bspns)
    rauc = roc_auc_score(y_true,y_pred)


    total_auc = M_s_auc + M_bpsns_auc + M_bspns_auc + rauc
    total_auc/= 4

    return total_auc

In [34]:
y_val=Y_n_v[:,[0, 1]]
y_val[:,0].shape, val_preds.shape,y_identities_val.shape

((90244,), (90244,), (90244, 24))

In [36]:
# mini-batch generator
def test_batch_iter(data, batch_size, shuffle=False):
    num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1
    print("batch_size", batch_size)
    print("num_batches_per_epoch", num_batches_per_epoch)

    def data_generator():
        data_size = len(data)

        while True:
            # Shuffle the data at each epoch
            if shuffle:
                shuffle_indices = np.random.permutation(np.arange(data_size))
                shuffled_data = data[shuffle_indices]
                #shuffled_labels = labels[shuffle_indices]
            else:
                shuffled_data = data
                #shuffled_labels = labels

            for batch_num in range(num_batches_per_epoch):
                start_index = batch_num * batch_size
                end_index = min((batch_num + 1) * batch_size, data_size)
                ###
                shuffled_D=shuffled_data[start_index: end_index]
                shuffled_D.tolist()
                shuffled_D = [' '.join(t.split()[0:time_steps]) for t in shuffled_D]
                shuffled_D = np.array(shuffled_D)                
                X_voc = get_voc_vec(shuffled_D, vocabulary)
                                
                sentence_split_list = []
                sentence_split_length_list = []
                
                

                for sentence in shuffled_D:
                    
                    
                    
                    sentence_split = sentence.split()
                    sentence_split_length = len(sentence_split)
                    sentence_split += ["NaN"] * (time_steps - sentence_split_length)
                    
                    sentence_split_list.append((" ").join(sentence_split))
                    sentence_split_length_list.append(sentence_split_length)
        
                X_elmo = np.array(sentence_split_list)

                X = [X_voc, X_elmo]
                
                #y = shuffled_labels[start_index: end_index]
                #y_train=y[:,[0, 1]]
                #y_aux_train=y[:,[2,3,4,5,6,7]]
                
                #Lables=[y_train,y_aux_train]
                #yield X,Lables
                yield X

    return num_batches_per_epoch, data_generator()

In [37]:
test_steps, test_batches = test_batch_iter(test['comment_text'],
                                        
                                        batch_size)


batch_size 256
num_batches_per_epoch 381


In [38]:
predictions = model.predict_generator(test_batches, steps=test_steps)[0].flatten()

In [42]:
df_submit = pd.read_csv('../input/sample_submission.csv')
df_submit.prediction = predictions
df_submit.to_csv('submission.csv', index=False)